## Importar das Bibliotecas

In [ ]:
import pandas as pd
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
import os

## Carregar documentos

### Carregar dados de leitura de caudal

In [ ]:
local = '' #nome do local
ano = #ano da análise

In [ ]:
Grafico = pd.read_excel(f'..\\data\\Reconstruídas\\{local.replace(' ','_')}_{ano}.xlsx')
Grafico_Outliers = pd.read_excel(f'..\\data\\Resultados\\{local.replace(' ','_')}_{ano}_Resultado.xlsx')
Anomalias = pd.read_excel('..\\data\\00 - Avarias.xlsx', sheet_name=f'{local.replace(' ','_')}')

In [ ]:
# Melt the dataframe to convert it to two columns: 'Date' and 'Value'
Grafico_Derretido = Grafico.melt(id_vars=['Date'], var_name='Hora', value_name='Caudal')
Grafico_Derretido['Hora'] = Grafico_Derretido['Hora'].str.replace('X', '', regex=False)
Grafico_Outliers_Derretido = Grafico_Outliers.melt(id_vars=['Date'], var_name='Hora', value_name='Anomalia')
Grafico_Outliers_Derretido['Hora'] = Grafico_Outliers_Derretido['Hora'].str.replace('X', '', regex=False)

#Porto_Carro_2016_Grafico_Derretido

In [ ]:
# Adicionar uma coluna de mês
Grafico_Derretido['Mês'] = Grafico_Derretido['Date'].dt.month

# Adicionar uma coluna de ano
Grafico_Derretido['Ano'] = Grafico_Derretido['Date'].dt.year

In [ ]:
# Converter a coluna 'Date' para datetime
Grafico_Outliers_Derretido['Date'] = pd.to_datetime(Grafico_Outliers_Derretido['Date'], format='%d/%m/%Y', dayfirst=True)

# Adicionar uma coluna de mês
Grafico_Outliers_Derretido['Mês'] = Grafico_Outliers_Derretido['Date'].dt.month

# Adicionar uma coluna de ano
Grafico_Outliers_Derretido['Ano'] = Grafico_Outliers_Derretido['Date'].dt.year

# Adicionar uma coluna de Caudal a tabela de Outliers
Grafico_Outliers_Derretido['Caudal'] = Grafico_Derretido['Caudal']

In [ ]:
# Group by 'Mês' and 'Hora' and calculate the required statistics
statistics = Grafico_Derretido.groupby(['Mês', 'Hora', 'Ano']).agg(
    Percentil_95=('Caudal', lambda x: np.percentile(x, 95)),
    Media=('Caudal', 'mean'),
    Mediana=('Caudal', 'median'),
    Percentil_5=('Caudal', lambda x: np.percentile(x, 5))
).reset_index()

#statistics

In [ ]:
# Filtrar os valores iguais a 1
Grafico_Anomalias_Comunicadas = Grafico_Outliers_Derretido[Grafico_Outliers_Derretido['Anomalia'] == 1]

#Porto_Carro_2016_Grafico_Anomalias_Comunicadas

In [ ]:
# Iterar sobre cada linha em Anomalia
nome = local

hora_index = ['00.00', '00.15', '00.30', '00.45', '01.00', '01.15', '01.30', '01.45', '02.00', '02.15', '02.30', '02.45', '03.00', '03.15', '03.30', '03.45', '04.00', '04.15', '04.30', '04.45', '05.00', '05.15', '05.30', '05.45', '06.00', '06.15', '06.30', '06.45', '07.00', '07.15', '07.30', '07.45', '08.00', '08.15', '08.30', '08.45', '09.00', '09.15', '09.30', '09.45', '10.00', '10.15', '10.30', '10.45', '11.00', '11.15', '11.30', '11.45', '12.00', '12.15', '12.30', '12.45', '13.00', '13.15', '13.30', '13.45', '14.00', '14.15', '14.30', '14.45', '15.00', '15.15', '15.30', '15.45', '16.00', '16.15', '16.30', '16.45', '17.00', '17.15', '17.30', '17.45', '18.00', '18.15', '18.30', '18.45', '19.00', '19.15', '19.30', '19.45', '20.00', '20.15', '20.30', '20.45', '21.00', '21.15', '21.30', '21.45', '22.00', '22.15', '22.30', '22.45', '23.00', '23:15', '23:30', '23,45']
hora_index_legend = ['00.00', '01.00', '02.00', '03.00', '04.00', '05.00', '06.00', '07.00', '08.00', '09.00', '10.00', '11.00', '12.00', '13.00', '14.00', '15.00', '16.00', '17.00', '18.00', '19.00', '20.00', '21.00', '22.00', '23.00']

for index, row in Anomalias.iterrows():
    data_inicio = row['Data/Hora']
    data_fim = row['Data Conclusão']
    fecho_agua = row['Fecho_de_agua']
    
    #print(row)
    
    # Filtrar as estatísticas para o mês e ano da anomalia
    mes_anomalia = data_inicio.month
    ano_anomalia = data_inicio.year
    mes_anomalia_fim = data_fim.month
    ano_anomalia_fim = data_fim.year
    
    hora_anomalia = data_inicio.hour * 4
    hora_anomalia_fim = data_fim.hour * 4
    minutos_anomalia = round(data_inicio.minute / 15)
    minutos_anomalia_fim = round(data_fim.minute / 15)
    
    stats_mes = statistics[(statistics['Mês'] == mes_anomalia) & (statistics['Ano'] == ano_anomalia)]
    
    if not stats_mes.empty:
        if mes_anomalia == mes_anomalia_fim:
            # Filtrar os dados de Porto_Carro_2016_Grafico_Derretido para o intervalo de tempo da anomalia
            dados_anomalia = Grafico_Anomalias_Comunicadas[Grafico_Anomalias_Comunicadas['Mês'] == mes_anomalia]
            # Separar os dados por dia
            dados_anomalia = dados_anomalia[(dados_anomalia['Date'].dt.day >= data_inicio.day) & (dados_anomalia['Date'].dt.day <= data_fim.day)]
            
            
            for dia in dados_anomalia['Date'].dt.day.unique():
                hora_index_temp = hora_index
                
                # Filtrar os dados de Porto_Carro_2016_Grafico_Derretido para cada dia da anomalia
                dados_anomalia_dia = dados_anomalia[dados_anomalia['Date'].dt.day == dia]
                
                if ((dia == data_inicio.day) and (dia == data_fim.day)):
                    # Filtrar os dados_anomalia para incluir apenas as linhas com 'Hora' em hora_index_temp
                    hora_index_temp = hora_index[int(hora_anomalia + np.floor(minutos_anomalia) - 1):int(hora_anomalia_fim + np.floor(minutos_anomalia_fim) - 1)]
                    dados_anomalia_dia = dados_anomalia_dia[dados_anomalia_dia['Hora'].isin(hora_index_temp)]
                
                elif (dia == data_inicio.day):
                    # Filtrar os dados_anomalia para incluir apenas as linhas com 'Hora' em hora_index_temp
                    hora_index_temp = hora_index_temp[(hora_anomalia + minutos_anomalia):]
                    dados_anomalia_dia = dados_anomalia_dia[dados_anomalia_dia['Hora'].isin(hora_index_temp)]
                    
                elif (dia == data_fim.day):
                    # Filtrar os dados_anomalia para incluir apenas as linhas com 'Hora' em hora_index_temp
                    hora_index_temp = hora_index_temp[:(hora_anomalia_fim + minutos_anomalia_fim)]
                    dados_anomalia_dia = dados_anomalia_dia[dados_anomalia_dia['Hora'].isin(hora_index_temp)]
                
                #Criar a figura
                plt.figure(figsize=(15, 7))
                
                # Plotar as estatísticas
                plt.plot(stats_mes['Hora'], stats_mes['Percentil_95'], label=f'Percentil 95 - Mês {mes_anomalia}', linestyle='-', marker='o')
                plt.plot(stats_mes['Hora'], stats_mes['Media'], label=f'Media - Mês {mes_anomalia}', linestyle='-', marker='x')
                plt.plot(stats_mes['Hora'], stats_mes['Mediana'], label=f'Mediana - Mês {mes_anomalia}', linestyle='-', marker='s')
                plt.plot(stats_mes['Hora'], stats_mes['Percentil_5'], label=f'Percentil 5 - Mês {mes_anomalia}', linestyle='-', marker='d')

            
                
                # Plotar os dados da anomalia
                if fecho_agua == True:
                    plt.plot(dados_anomalia_dia['Hora'], dados_anomalia_dia['Caudal'], color='purple', label=f'Anomalia - Dia {dia}', linestyle='--', marker='3')
                else:
                    plt.plot(dados_anomalia_dia['Hora'], dados_anomalia_dia['Caudal'], color='cyan', label=f'Anomalia - Dia {dia}', linestyle='--', marker='3')
                
                # Adicionar título e rótulos
                plt.title(f'Estatísticas e Anomalia para {nome} em {data_inicio.year} - {data_inicio.strftime("%d/%m/%Y %H:%M:%S")} a {data_fim.strftime("%d/%m/%Y %H:%M:%S")}', pad=20)
                plt.xlabel('Hora')
                plt.xticks(ticks=range(0, len(hora_index), 4), labels=[hora for i, hora in enumerate(hora_index) if i % 4 == 0])
                plt.ylabel('Caudal')
                plt.grid(True, alpha=0.5)
                
                # Set legend background with transparency
                plt.legend(frameon=True, framealpha=0.3)
                
                # Defenir a cor de fundo e transparência
                plt.gca().set_facecolor('none')  # Set the axes background to be transparent
                plt.gcf().set_facecolor('none')  # Set the figure background to be transparent
                plt.gcf().set_alpha(0)  # Set the figure background to be transparent

                # Criar uma pasta para guardar os gráficos
                os.makedirs(f'..\\data\\Gráficos\\Anomalias Individuais\\{nome}\\{ano_anomalia}\\Estatísticas_Anomalia_{data_inicio.strftime("%Y-%m-%d")}-{data_fim.strftime("%Y-%m-%d")}_{index}', exist_ok=True)
                
                # Save the plot in different formats in a different directory with higher resolution
                plt.savefig(f'..\\data\\Gráficos\\Anomalias Individuais\\{nome}\\{ano_anomalia}\\Estatísticas_Anomalia_{data_inicio.strftime("%Y-%m-%d")}-{data_fim.strftime("%Y-%m-%d")}_{index}\\Anomalia - Dia {dia}.svg', dpi=10000)
                plt.savefig(f'..\\data\\Gráficos\\Anomalias Individuais\\{nome}\\{ano_anomalia}\\Estatísticas_Anomalia_{data_inicio.strftime("%Y-%m-%d")}-{data_fim.strftime("%Y-%m-%d")}_{index}\\Anomalia - Dia {dia}.png', dpi=500)
                plt.savefig(f'..\\data\\Gráficos\\Anomalias Individuais\\{nome}\\{ano_anomalia}\\Estatísticas_Anomalia_{data_inicio.strftime("%Y-%m-%d")}-{data_fim.strftime("%Y-%m-%d")}_{index}\\Anomalia - Dia {dia}.tiff', dpi=500)
                
                plt.show()
                
        else:
            # Filtrar os dados de Porto_Carro_2016_Grafico_Derretido para o intervalo de tempo da anomalia
            dados_anomalia = Grafico_Anomalias_Comunicadas[(Grafico_Anomalias_Comunicadas['Mês'] >= mes_anomalia) & (Grafico_Anomalias_Comunicadas['Mês'] <= mes_anomalia_fim)]
            # Separar os dados por dia
            dados_anomalia = dados_anomalia[(dados_anomalia['Date'].dt.month >= mes_anomalia) & (dados_anomalia['Date'].dt.month <= mes_anomalia_fim)]
            dados_anomalia_temp_inicio = dados_anomalia[((dados_anomalia['Date'].dt.day >= data_inicio.day) & (dados_anomalia['Date'].dt.month >= mes_anomalia))]
            dados_anomalia_temp_fim = dados_anomalia[((dados_anomalia['Date'].dt.day <= data_fim.day) & (dados_anomalia['Date'].dt.month <= mes_anomalia_fim))]
            
            dados_anomalia = pd.concat([dados_anomalia_temp_inicio, dados_anomalia_temp_fim])
            
            for date in dados_anomalia['Date'].dt.date.unique():
                month_chose = 0
                
                if (date.month != mes_anomalia):
                    month_chose = date.month - mes_anomalia
                
                dia = date.day
                
                hora_index_temp = hora_index
                
                # Filtrar os dados de Porto_Carro_2016_Grafico_Derretido para cada dia da anomalia
                dados_anomalia_dia = dados_anomalia[dados_anomalia['Date'].dt.day == dia]
                
                if ((dia == data_inicio.day) and (dia == data_fim.day)):
                    # Filtrar os dados_anomalia para incluir apenas as linhas com 'Hora' em hora_index_temp
                    hora_index_temp = hora_index[int(hora_anomalia + np.floor(minutos_anomalia) - 1):int(hora_anomalia_fim + np.floor(minutos_anomalia_fim) - 1)]
                    dados_anomalia_dia = dados_anomalia_dia[dados_anomalia_dia['Hora'].isin(hora_index_temp)]
                
                elif (dia == data_inicio.day):
                    # Filtrar os dados_anomalia para incluir apenas as linhas com 'Hora' em hora_index_temp
                    hora_index_temp = hora_index_temp[(hora_anomalia + minutos_anomalia):]
                    dados_anomalia_dia = dados_anomalia_dia[dados_anomalia_dia['Hora'].isin(hora_index_temp)]
                    
                elif (dia == data_fim.day):
                    # Filtrar os dados_anomalia para incluir apenas as linhas com 'Hora' em hora_index_temp
                    hora_index_temp = hora_index_temp[:(hora_anomalia_fim + minutos_anomalia_fim)]
                    dados_anomalia_dia = dados_anomalia_dia[dados_anomalia_dia['Hora'].isin(hora_index_temp)]
                
                #Criar a figura
                plt.figure(figsize=(15, 7))
                
                if (month_chose != 0):
                    # Plotar as estatísticas
                    plt.plot(statistics[(statistics['Mês'] == (mes_anomalia + month_chose)) & (statistics['Ano'] == ano_anomalia)]['Hora'], statistics[(statistics['Mês'] == (mes_anomalia + month_chose)) & (statistics['Ano'] == ano_anomalia)]['Percentil_95'], label=f'Percentil 95 - Mês {(mes_anomalia + month_chose)}', linestyle='-', marker='o')
                    plt.plot(statistics[(statistics['Mês'] == (mes_anomalia + month_chose)) & (statistics['Ano'] == ano_anomalia)]['Hora'], statistics[(statistics['Mês'] == (mes_anomalia + month_chose)) & (statistics['Ano'] == ano_anomalia)]['Media'], label=f'Media - Mês {(mes_anomalia + month_chose)}', linestyle='-', marker='x')
                    plt.plot(statistics[(statistics['Mês'] == (mes_anomalia + month_chose)) & (statistics['Ano'] == ano_anomalia)]['Hora'], statistics[(statistics['Mês'] == (mes_anomalia + month_chose)) & (statistics['Ano'] == ano_anomalia)]['Mediana'], label=f'Mediana - Mês {(mes_anomalia + month_chose)}', linestyle='-', marker='s')
                    plt.plot(statistics[(statistics['Mês'] == (mes_anomalia + month_chose)) & (statistics['Ano'] == ano_anomalia)]['Hora'], statistics[(statistics['Mês'] == (mes_anomalia + month_chose)) & (statistics['Ano'] == ano_anomalia)]['Percentil_5'], label=f'Percentil 5 - Mês {(mes_anomalia + month_chose)}', linestyle='-', marker='d')
                
                else:
                    # Plotar as estatísticas
                    plt.plot(stats_mes['Hora'], stats_mes['Percentil_95'], label=f'Percentil 95 - Mês {mes_anomalia}', linestyle='-', marker='o')
                    plt.plot(stats_mes['Hora'], stats_mes['Media'], label=f'Media - Mês {mes_anomalia}', linestyle='-', marker='x')
                    plt.plot(stats_mes['Hora'], stats_mes['Mediana'], label=f'Mediana - Mês {mes_anomalia}', linestyle='-', marker='s')
                    plt.plot(stats_mes['Hora'], stats_mes['Percentil_5'], label=f'Percentil 5 - Mês {mes_anomalia}', linestyle='-', marker='d')
                
                # Plotar os dados da anomalia
                if fecho_agua == True:
                    plt.plot(dados_anomalia_dia['Hora'], dados_anomalia_dia['Caudal'], color='purple', label=f'Anomalia - Dia {dia}', linestyle='--', marker='3')
                else:
                    plt.plot(dados_anomalia_dia['Hora'], dados_anomalia_dia['Caudal'], color='cyan', label=f'Anomalia - Dia {dia}', linestyle='--', marker='3')
                
                # Adicionar título e rótulos
                #plt.title(f'Estatísticas e Anomalia para {nome} em {data_inicio.year} - {data_inicio.strftime("%d/%m/%Y %H:%M:%S")} a {data_fim.strftime("%d/%m/%Y %H:%M:%S")}', pad=20)
                plt.xlabel('Hora')
                plt.xticks(ticks=range(0, len(hora_index), 4), labels=[hora for i, hora in enumerate(hora_index) if i % 4 == 0])
                plt.ylabel('Caudal')
                plt.legend()
                #plt.xticks(rotation=90)
                plt.grid(True, alpha=0.5)
                
                # Set legend background with transparency
                plt.legend(frameon=True, facecolor='none', framealpha=0.5)
                
                # Defenir a cor de fundo e transparência
                plt.gca().set_facecolor('none')  # Set the axes background to be transparent
                plt.gcf().set_facecolor('none')  # Set the figure background to be transparent
                plt.gcf().set_alpha(0)  # Set the figure background to be transparent

                # Criar uma pasta para guardar os gráficos
                os.makedirs(f'..\\data\\Gráficos\\Anomalias Individuais\\{nome}\\{ano_anomalia}\\Estatísticas_Anomalia_{data_inicio.strftime("%Y-%m-%d")}-{data_fim.strftime("%Y-%m-%d")}_{index}', exist_ok=True)
                
                # Save the plot in different formats in a different directory with higher resolution
                plt.savefig(f'..\\data\\Gráficos\\Anomalias Individuais\\{nome}\\{ano_anomalia}\\Estatísticas_Anomalia_{data_inicio.strftime("%Y-%m-%d")}-{data_fim.strftime("%Y-%m-%d")}_{index}\\Anomalia - Dia {dia} - Mês {(mes_anomalia + month_chose)}.svg', dpi=10000)
                plt.savefig(f'..\\data\\Gráficos\\Anomalias Individuais\\{nome}\\{ano_anomalia}\\Estatísticas_Anomalia_{data_inicio.strftime("%Y-%m-%d")}-{data_fim.strftime("%Y-%m-%d")}_{index}\\Anomalia - Dia {dia} - Mês {(mes_anomalia + month_chose)}.png', dpi=500)
                plt.savefig(f'..\\data\\Gráficos\\Anomalias Individuais\\{nome}\\{ano_anomalia}\\Estatísticas_Anomalia_{data_inicio.strftime("%Y-%m-%d")}-{data_fim.strftime("%Y-%m-%d")}_{index}\\Anomalia - Dia {dia} - Mês {(mes_anomalia + month_chose)}.tiff', dpi=500)
                
                plt.show()
        
        